In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation

from datetime import datetime

# **DATA**

In [15]:
new_market = pd.read_csv('./new_market.csv')

In [16]:
ticker_15 = ['AAPL', 'AMZN', 'NVDA', 'JPM', 'PG', 'PFE', 'JNJ', 'KO', 'XOM', 'NEE',
             'GOOGL', 'MSFT', 'TSLA', 'NKE', 'BAC']

ticker_30 = ['MSFT', 'AMZN', 'NVDA', 'AAPL', 'GOOGL', 'ADBE', 'JNJ', 'PFE', 'MRK', 
             'ABT', 'PG', 'KO', 'JPM', 'GS', 'CAT', 'CVX', 'XOM', 'BA', 'TSLA', 'NEE', 
             'NKE', 'VZ', 'CRM', 'UNH', 'WMT', 'QCOM', 'BAC', 'V', 'MCD', 'INTC']

ticker_45 = ['AAPL', 'GOOGL', 'MSFT', 'NVDA', 'AMD', 'ORCL', 'CRM', 'INTC', 'CSCO', 
             'JPM', 'GS', 'BAC', 'MS', 'AXP', 'C', 'JNJ', 'UNH', 'PFE', 'ABBV', 'TSLA', 
             'AMGN', 'GILD', 'PG', 'KO', 'NKE', 'PEP', 'COST', 'WMT', 'TGT', 'XOM', 
             'CVX', 'NEE', 'DUK', 'SO', 'SLB', 'MMM', 'CAT', 'HON', 'GE', 'ADP', 'AMZN', 
             'META', 'HD', 'VZ', 'MRK']

a = ['pricingDate'] + ticker_15
b = ['pricingDate'] + ticker_30
c = ['pricingDate'] + ticker_45
 
ticker_15_price = new_market[a]
ticker_30_price = new_market[b]
ticker_45_price = new_market[c]

In [18]:
ticker_15_price.to_csv('./ticker_15_price.csv')
ticker_30_price.to_csv('./ticker_30_price.csv')
ticker_45_price.to_csv('./ticker_45_price.csv')

# **GPT-weighted Portfolio simulation**

In [30]:
gpt_weighted_15 = {
    "AAPL": 0.084, "AMZN": 0.074, "NVDA": 0.064, "JPM": 0.054, "PG": 0.054,
    "PFE": 0.064, "JNJ": 0.064, "KO": 0.054, "XOM": 0.064, "NEE": 0.074,
    "GOOGL": 0.084, "MSFT": 0.084, "TSLA": 0.074, "NKE": 0.054, "BAC": 0.054
}

gpt_weighted_30 = {
    "MSFT": 0.067, "AMZN": 0.067, "NVDA": 0.067, "AAPL": 0.067, "GOOGL": 0.067,
    "ADBE": 0.067, "JNJ": 0.05, "PFE": 0.05, "MRK": 0.05, "ABT": 0.05, 
    "PG": 0.05, "KO": 0.05, "JPM": 0.05, "GS": 0.05, "CAT": 0.025, 
    "CVX": 0.025, "XOM": 0.025, "BA": 0.025, "TSLA": 0.025, "NEE": 0.025, 
    "NKE": 0.005, "VZ": 0.005, "CRM": 0.005, "UNH": 0.005, "WMT": 0.005, 
    "QCOM": 0.005, "BAC": 0.005, "V": 0.005, "MCD": 0.005, "INTC": 0.005
}

gpt_weighted_45 = {
    "AAPL": 0.05, "GOOGL": 0.04, "MSFT": 0.04, "NVDA": 0.03, "AMD": 0.03, 
    "ORCL": 0.02, "CRM": 0.02, "INTC": 0.01, "CSCO": 0.01, "JPM": 0.04, 
    "GS": 0.03, "BAC": 0.03, "MS": 0.02, "AXP": 0.02, "C": 0.01,
    "JNJ": 0.03, "UNH": 0.03, "PFE": 0.02, "ABBV": 0.02, "TSLA": 0.02, 
    "AMGN": 0.02, "GILD": 0.01, "PG": 0.03, "KO": 0.03, "NKE": 0.02, 
    "PEP": 0.02, "COST": 0.02, "WMT": 0.02, "TGT": 0.01, "XOM": 0.025, 
    "CVX": 0.025, "NEE": 0.02, "DUK": 0.01, "SO": 0.01, "SLB": 0.01,
    "MMM": 0.02, "CAT": 0.02, "HON": 0.02, "GE": 0.02, "ADP": 0.02,
    "AMZN": 0.02, "META": 0.02, "HD": 0.02, "VZ": 0.01, "MRK": 0.01
}

In [20]:
# 전처리
# 'pricingDate' to datetime
ticker_15_price['pricingDate'] = pd.to_datetime(ticker_15_price['pricingDate'])

# 5년기간 정의(2018-05-01 to 2023-05-01)
start_date = datetime(2018, 5, 1)
end_date = datetime(2023, 5, 1)

filtered_data = ticker_15_price[(ticker_15_price['pricingDate'] >= start_date) & (ticker_15_price['pricingDate'] <= end_date)]

filtered_data.head(), filtered_data.shape

/var/folders/vk/4yxtbn3d26j6dby6kfglqvbr0000gp/T/ipykernel_1448/2688966448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_15_price['pricingDate'] = pd.to_datetime(ticker_15_price['pricingDate'])


(  pricingDate       AAPL      AMZN       NVDA        JPM         PG  \
 0  2018-05-01  40.191708  79.11300  56.227008  92.320306  62.678005   
 1  2018-05-02  41.967178  78.48400  56.021547  91.590434  61.789573   
 2  2018-05-03  42.043236  78.60375  57.675137  91.013326  62.155398   
 3  2018-05-04  43.692736  79.04750  59.177726  92.023265  63.087380   
 4  2018-05-07  44.008850  80.00700  61.559093  92.821032  62.695425   
 
          PFE         JNJ         KO        XOM        NEE    GOOGL       MSFT  \
 0  27.354391  108.702901  35.810358  58.326616  36.037792  52.0375  89.274061   
 1  26.960303  106.537642  35.364726  58.212919  36.088574  51.3025  87.873868   
 2  26.859849  106.132195  35.221787  58.015844  36.002465  51.3150  88.400115   
 3  26.921667  107.132872  35.616971  58.288717  36.229882  52.5500  89.424418   
 4  26.991212  106.615280  35.431991  58.925421  36.066495  52.9730  90.420528   
 
         TSLA        NKE        BAC  
 0  19.994666  64.398101  26.34551

In [26]:
# 2018.05.01 ~ 2023.04.30 기간에서 30일씩 3번에 걸쳐서 랜덤하게 선택되는 시나리오를 1000개 만들기

# 각 시나리오 부분별로 필요한 일수
days_per_part = 30

num_scenarios = 1000
total_days = ticker_15_price.shape[0]

scenarios = []

# 각 시나리오를 3개의 랜덤한 30일로 구성

for _ in range(num_scenarios):
   
    scenario_data = pd.DataFrame()

    # 사용된 일자를 추적하기 위한 집합
    used_days = set()

    for _ in range(3):
        # 중복되지 않는 30일 기간을 선택
        while True:
            start_day = np.random.randint(0, total_days - days_per_part)
            end_day = start_day + days_per_part

            # 선택된 기간이 이전에 사용된 일자와 중복되지 않는지 확인
            if not used_days.intersection(set(range(start_day, end_day))):
                break

        # 사용된 일자 업데이트
        used_days.update(range(start_day, end_day))

        # 데이터셋에서 30일 기간의 데이터 추출
        part_data = ticker_15_price.iloc[start_day:end_day]

        # 추출된 30일 데이터를 시나리오 데이터에 추가
        scenario_data = pd.concat([scenario_data, part_data])

    # 완성된 시나리오 데이터를 시나리오 목록에 추가
    scenarios.append(scenario_data)


scenarios[0]


,pricingDate,AAPL,AMZN,NVDA,JPM,PG,PFE,JNJ,KO,XOM,NEE,GOOGL,MSFT,TSLA,NKE,BAC
1255,2023-04-26,163.313267,104.9800,269.509898,133.323854,153.612448,37.795378,160.238800,62.573226,113.526185,73.092330,103.7100,294.103171,153.750000,123.021074,27.981674
1256,2023-04-27,167.950582,109.8200,272.209396,135.118200,155.490618,37.903001,160.613236,62.701228,114.883189,74.848835,107.5900,303.522597,160.190000,124.868774,28.424422
1257,2023-04-28,169.217117,105.4500,277.438424,136.291426,155.401181,38.049760,161.302986,63.164004,116.368027,75.618540,107.3400,305.942175,164.310000,125.882029,28.808137
1258,2023-05-01,169.127363,102.0500,289.046266,139.209703,155.589992,38.362846,161.204450,63.311699,112.759183,75.786296,107.2000,304.249466,161.830000,127.074094,28.572004
1259,2023-05-02,168.080227,103.6300,282.047567,136.961841,155.450868,38.216087,162.613511,63.026156,108.265335,74.937647,105.3200,304.100109,160.310000,126.418458,27.706186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,2022-02-09,174.740588,161.1895,266.705621,148.297901,153.661365,48.697733,163.477099,57.930410,74.970886,73.593335,141.5920,306.231588,310.666666,143.339669,47.077224
953,2022-02-10,170.616916,159.0035,257.906982,147.682361,151.321784,47.874593,161.503820,58.253089,74.249647,72.431739,138.6200,297.542841,301.516666,141.928421,46.857505
954,2022-02-11,167.167306,153.2935,239.181161,145.759981,150.474528,48.044898,159.873720,57.218617,76.119174,71.961341,134.2825,290.320259,286.666666,137.381066,45.778015
955,2022-02-14,167.405210,155.1670,242.357060,144.405792,150.907784,47.117682,157.862310,57.588749,74.951906,71.922941,135.5260,290.280899,291.920000,138.762913,45.300365


In [28]:
# 시나리오가 랜덤하게 잘 구성되었는지 확인

scenario0 = scenarios[0]
scenario1 = scenarios[1]
scenario2 = scenarios[2]

In [33]:
# 각 시나리오에 대해 주어진 가중치를 적용하여 포트폴리오의 성과를 평가

# 포트폴리오 성과를 계산하는 함수
def calculate_portfolio_performance(scenario, weights):
    # 시작일과 종료일의 가격을 추출
    start_prices = scenario.iloc[0][list(weights.keys())]
    end_prices = scenario.iloc[-1][list(weights.keys())]

    # 가중치에 따라 포트폴리오의 성과를 계산
    performance = ((end_prices - start_prices) / start_prices) * list(weights.values())
    return performance.sum()

# 각 시나리오에 대한 포트폴리오 성과를 계산
portfolio_performances = [calculate_portfolio_performance(scenario, gpt_weighted_15) for scenario in scenarios]

portfolio_performances[:10]

[0.17555321142446403,
 0.03155087928249085,
 0.19296092325797662,
 2.462864275785011,
 1.8920800926803132,
 0.020741088356068672,
 -0.3895674749255519,
 1.6589701804925834,
 -0.393532023702788,
 -0.4411618964324232]

# **Mean-Variance Optimization**

In [9]:
new_market = pd.read_csv('./new_market.csv', index_col=0)
new_market

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,63.574105,42.342797,40.191708,79.940979,53.921780,26.876666,140.357973,224.08,79.992862,38.801281,...,180.343060,39.841488,58.326616,47.887170,65.827974,78.555893,108.770822,136.78,46.462606,80.580399
2018-05-02,63.257386,41.249072,41.967178,78.609543,53.032556,26.310000,139.240061,221.10,79.488744,38.284275,...,179.572242,39.747543,58.212919,47.839888,64.434146,72.716789,109.245154,133.12,46.580767,77.198727
2018-05-03,63.670080,41.122121,42.043236,78.539055,53.105894,25.983333,140.182433,226.05,79.137661,37.569084,...,180.126855,39.696300,58.015844,46.563275,65.135737,74.956321,107.124611,133.54,45.956201,77.662499
2018-05-04,64.303518,41.854527,43.692736,78.452903,53.848442,26.276666,141.762291,228.51,81.262162,37.793120,...,180.869471,39.619435,58.288717,47.272504,65.556692,74.738715,108.091876,134.67,46.808649,78.802606
2018-05-07,64.677822,41.541303,44.008850,78.029977,54.380142,26.376666,140.709052,230.99,81.469211,37.646635,...,180.399461,39.397382,58.925421,44.369392,65.500564,75.355266,108.808024,137.48,46.850849,79.372659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,111.670000,12.730000,173.660000,147.690000,95.650000,80.460000,308.112102,518.42,173.790000,73.470000,...,88.210000,56.630000,111.500000,33.300000,91.220000,123.530000,109.690000,230.94,33.330000,170.990000
2023-10-05,110.350000,12.850000,174.910000,147.450000,96.200000,81.640000,308.380987,516.44,172.100000,73.130000,...,89.490000,56.480000,108.990000,32.180000,90.140000,121.840000,109.300000,222.54,33.400000,172.000000
2023-10-06,110.640000,12.760000,177.490000,148.240000,96.880000,82.180000,310.900537,526.68,173.970000,72.860000,...,92.410000,57.350000,107.170000,32.480000,90.970000,119.460000,110.910000,223.85,33.690000,175.580000


In [7]:
new_market['pricingDate'] = pd.to_datetime(new_market['pricingDate'])
new_market.set_index('pricingDate', inplace=True)

# 평균, 공분산 계산
mu = expected_returns.mean_historical_return(new_market)
Sigma = risk_models.sample_cov(new_market)

# Efficient Frontier
ef = EfficientFrontier(mu, Sigma)
weights = ef.min_volatility()


cleaned_weights = ef.clean_weights()
print("Optimized Portfolio Weights:")
print(cleaned_weights)

# 최적화
latest_prices = new_market.iloc[-1]  # Use the latest prices
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.lp_portfolio()
allocation, leftover

Optimized Portfolio Weights:
OrderedDict([('A', 0.0), ('AAL', 0.0), ('AAPL', 0.0), ('ABBV', 0.02224), ('ABT', 0.0), ('ACGL', 0.0), ('ACN', 0.0), ('ADBE', 0.0), ('ADI', 0.0), ('ADM', 0.0), ('ADP', 0.0), ('ADSK', 0.0), ('AEE', 0.0), ('AEP', 0.0), ('AES', 0.0), ('AFL', 0.0), ('AIG', 0.0), ('AIZ', 0.0), ('AJG', 0.0), ('AKAM', 0.0), ('ALB', 0.0), ('ALGN', 0.0), ('ALK', 0.0), ('ALL', 0.0), ('ALLE', 0.0), ('AMAT', 0.0), ('AMD', 0.0), ('AME', 0.0), ('AMGN', 0.0), ('AMP', 0.0), ('AMT', 0.0), ('AMZN', 0.0), ('ANET', 0.0), ('ANSS', 0.0), ('AON', 0.0), ('AOS', 0.0), ('APA', 0.0), ('APD', 0.0), ('APH', 0.0), ('APTV', 0.0), ('ARE', 0.0), ('ATO', 0.0), ('AVB', 0.0), ('AVGO', 0.0), ('AVY', 0.0), ('AWK', 0.0), ('AXON', 0.0), ('AXP', 0.0), ('AZO', 0.0), ('BA', 0.0), ('BAC', 0.0), ('BALL', 0.0), ('BAX', 0.0), ('BBWI', 0.0), ('BBY', 0.0), ('BDX', 0.0), ('BEN', 0.0), ('BF.B', 0.0), ('BG', 0.0), ('BIO', 0.0), ('BK', 0.0), ('BKNG', 0.0), ('BKR', 0.0), ('BLK', 0.0), ('BMY', 0.06595), ('BR', 0.0), ('BRO', 0.0)

In [ ]:
# 아래는 예시로 대략적으로 작성한 가상의 데이터입니다.

raw_price_data = {
    'AAPL': [150, 153, 151, 152, 155],
    'GOOGL': [2800, 2820, 2810, 2850, 2900],
    'MSFT': [300, 305, 302, 303, 310],
    # ... 다른 주식 데이터 ...
}

# 데이터프레임으로 변환합니다.
df = pd.DataFrame(raw_price_data)

# 수익률과 공분산 행렬 계산
mu = expected_returns.mean_historical_return(df)
Sigma = risk_models.sample_cov(df)

# 포트폴리오 최적화
ef = EfficientFrontier(mu, Sigma)
weights = ef.min_volatility()

# 최적화된 결과 출력
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

# 포트폴리오 할당량 계산
latest_prices = df.iloc[-1]  # 최신 주가를 사용하여 할당량 계산
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.lp_portfolio()
print("포트폴리오 할당량:", allocation)
print("잔여 현금:", leftover)